In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import time


devices = ("mps", "cpu")
batch_size = (200,)

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

    return correct


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def test_model_time(model, device: str, batch_size: int):
    """Time the model on different devices and batch sizes."""
    print(f"Device: {device}")
    print(f"Batch size: {batch_size}")

    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 20
    start_time = time.time()
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
    accuracy = test(test_dataloader, model, loss_fn)
    print("Done!")
    end_time = time.time()
    print(f"Time: {end_time - start_time:.2f} s\n")

    # Write results to file
    with open("results.txt", "a") as f:
        string = f"Device: {device}, Batch size: {batch_size}, Time: {end_time - start_time:.2f}s, Accuracy: {(100*accuracy):>0.1f}%, Epochs: {epochs}\n\n"
        f.write(string)


for device in devices:
    for bs in batch_size:
        train_dataloader = DataLoader(training_data, batch_size=bs)
        test_dataloader = DataLoader(test_data, batch_size=bs)
        model = NeuralNetwork()
        test_model_time(model, device, bs)

Device: mps
Batch size: 200
Epoch 1
-------------------------------
loss: 2.298910  [  200/60000]
loss: 2.291062  [20200/60000]
loss: 2.269921  [40200/60000]
Epoch 2
-------------------------------
loss: 2.256307  [  200/60000]
loss: 2.249967  [20200/60000]
loss: 2.223465  [40200/60000]
Epoch 3
-------------------------------
loss: 2.209537  [  200/60000]
loss: 2.203354  [20200/60000]
loss: 2.170756  [40200/60000]
Epoch 4
-------------------------------
loss: 2.154755  [  200/60000]
loss: 2.148213  [20200/60000]
loss: 2.108306  [40200/60000]
Epoch 5
-------------------------------
loss: 2.086776  [  200/60000]
loss: 2.080281  [20200/60000]
loss: 2.031408  [40200/60000]
Epoch 6
-------------------------------
loss: 2.000274  [  200/60000]
loss: 1.995679  [20200/60000]
loss: 1.937592  [40200/60000]
Epoch 7
-------------------------------
loss: 1.893682  [  200/60000]
loss: 1.894108  [20200/60000]
loss: 1.828581  [40200/60000]
Epoch 8
-------------------------------
loss: 1.771691  [  200